# Week 18 - MQTT

**MQTT** stands for **Message Queuing Telemetry Transport**. It’s a lightweight, publish-subscribe messaging protocol designed for environments with low bandwidth, high latency, or unreliable networks, making it ideal for IoT (Internet of Things) applications.

<div>
  <span style="color: #0C8EE0; font-size: 20px;">
    <strong>Key characteristics:</strong>
  </span>
</div>

* **Lightweight:** Minimal overhead, suitable for small devices like sensors.
* **Efficient**: Optimised for constrained devices and networks.
* **Scalable:** Many devices can connect to a single broker and share data through topics.
* **Open Standard:** Maintained by OASIS and ISO (ISO/IEC 20922).
  
MQTT lets devices send and receive messages efficiently without talking directly to each other.

It is widely used in:

* Smart homes
* IoT sensors
* Industrial monitoring
* Cloud-connected microcontrollers (ESP32, Pico W, etc.)

#### How MQTT Works
MQTT uses a publish-subscribe architecture:

* **Broker**: Central server that manages message distribution.
* **Clients:** Devices that either publish messages to topics or subscribe to topics to receive messages.
* **Topics:** Hierarchical strings (e.g., `home/livingroom/temperature`) used to organise messages.
* **Transport:** Runs over TCP/IP for reliable, ordered delivery.
* Quality of Service (QoS):
    * 0: At most once (fast, no guarantee)
    * 1: At least once (guaranteed, may duplicate)
    * 2: Exactly once (guaranteed, no duplicates)
      
**The three main parts of MQTT**

**1. Publisher**

A device that sends data.

**Examples:**

* Temperature sensor
* RFID reader
* Button device

**2. Subscriber**

A device or application that receives data.

**Examples:**

* Dashboard
* LED controller
* Mobile app
* Another microcontroller

**3. Broker (the Middleman / Server)**

The middle server that:
* Receives messages
* Forwards them to subscribers.

**Examples of brokers:**
* **Mosquitto** (open-source, very popular).
* **HiveMQ** (cloud-based, great for learning).
* **EMQX** (enterprise broker for large-scale IoT)
  
📌 Devices never talk directly to each other — everything goes through the broker.


<div>
  <span style="color: #0C8EE0; font-size: 20px;">
    <strong>Topic in MQTT</strong>
  </span>
</div>

In MQTT, a topic is a named channel that messages are sent to and received from.

A topic tells the MQTT broker where a message belongs and who should receive it.

Devices publish messages to topics, and other devices subscribe to topics.

<div>
  <span style="color: #0C8EE0; font-size: 20px;">
    <strong>What does a topic look like?</strong>
  </span>
</div>

A topic is a text string, usually written in levels separated by **`/`**.

**Example**
````Bash
classroom/temperature
office/attendance/rfid
home/livingroom/light

````
Each **`/`** creates a level, like folders.

**Example flow**

````text
Pico publishes ➡️ "classroom/temp" : 24.5
Broker receives
Dashboard subscribes ➡️ "classroom/temp"
Dashboard gets 24.5
````

**The publisher and subscriber never talk directly — the topic connects them.**

<div>
  <span style="color: #0C8EE0; font-size: 20px;">
    <strong>What Happens When Data Is Sent</strong>
  </span>
</div>


1. Device connects to the MQTT broker.
2. Device publishes a message to a topic.
3. Broker receives the message.
4. Broker sends the message to all subscribers of that topic.
5. Subscribers react instantly.

⚡⚡**This happens in milliseconds**



<b><span style="color: darkblue; font-size: 20px;">Why MQTT Is Perfect for IoT:</span></b>

<table style="margin-left: 0; border-collapse: collapse; width: 50%; font-size: 14px;">
    <tr style="background-color: #13BDE3;">
       <th style="text-align: left;"><b>Feature</b></th>
        <th style="text-align: left;"><b>Why it matters</b></th>
    </tr>
    <tr>
        <td><b>Lightweight </b></td>
        <td>Ideal for Wi-Fi / mobile networks  </td>
    </tr>
    <tr>
        <td><b>Fast</b></td>
        <td>Near real-time updates  </td>
    </tr>
    <tr>
        <td><b>Scalable</b></td>
        <td> 1 → 1,000+ devices   </td>
    </tr>
    <tr>
        <td><b>Reliable options</b></td>
        <td>QoS levels      </td>
    </tr>
    <tr>
        <td><b>Decoupled</b></td>
        <td>Devices don’t know each other   </td>
    </tr>
</table>

<div>
  <span style="color: #0C8EE0; font-size: 20px;">
    <strong>Setting up MQTT with HiveMQ Cloud (with TLS Security)</strong>
  </span>
</div>


HiveMQ provides a free cloud-based broker that’s secure, reliable, and ideal for small IoT projects. Unlike running a broker locally (like Mosquitto), HiveMQ Cloud takes care of hosting and TLS security for you.

#### Why TLS with HiveMQ Cloud?
TLS (Transport Layer Security) ensures:

* **Encrypted communication** between MQTT clients and the HiveMQ broker.
* **Authentication** via certificates to prevent man-in-the-middle attacks.
* **Data integrity** so messages aren’t tampered with.

**Step 1: Create a HiveMQ Cloud Account**
1. Go to HiveMQ Cloud
2. Click **Sign Up** and create a free account (choose the free tier plan).
3. Confirm your email and log in to the HiveMQ Cloud Console.
    
**Step 2: Create a Cluster (Your Broker)**
1. Once logged in, click **Create Cluster**.
2. Choose:
    - Region: (e.g., Europe or US – closer region = faster connection).
    - Tier: Select Free.
3. HiveMQ will set up your cluster (broker) (e.g., `xxxx.s1.eu.hivemq.cloud`).
   
**Step 3: Create MQTT Credentials**
1. In the cluster settings, go to Access Management → MQTT Clients.
2. Click Create Credentials.
3.  Set:
    - **Username**: `pico_user1` (you choose).
    - **Password**: `StrongPassword123!` (use something strong).
4. Save – you’ll use these in your Pico W code to connect securely.

   
**Step 4: Enable TLS (Secure Communication)**
HiveMQ Cloud requires TLS/SSL encryption on port 8883.
This means:
- You must connect using port 8883 (not 1883).
- In MicroPython, the `umqtt.simple` library supports SSL with a little extra setup.
  
**TLS = Transport Layer Security.**
It ensures communication between your device and HiveMQ is:
- Encrypted → outsiders can’t read it.
- Authenticated → your Pico knows it’s talking to HiveMQ (not a fake broker).
- Tamper-proof → messages can’t be changed secretly on the way.

**Example:**
TLS is like the **padlock icon**  you see in your web browser with `https://`.
For MQTT, it means data travels in a secure tunnel.

**Step 5: Install MQTT Libraries on the Pico**

Your Raspberry Pi Pico W needs the MQTT libraries before running the code.
Use MicroPython’s package installer (mip) to fetch them:

```python
import mip

# Install basic MQTT client
mip.install("umqtt.simple")

# Install robust MQTT client (auto reconnect if Wi-Fi drops)
mip.install("umqtt.robust")

```
<div>
  <span style="color: red; font-size: 15px;">
    <strong>Run the above code once in Thonny (with Wi-Fi connected to your pico).</strong>
  </span>
</div>

It will download and save the libraries into your Pico’s `/lib` folder.

- Use `umqtt.simple`.

- Use `umqtt.robust` for projects that need auto-reconnects.

<div>
  <span style="color: #0C8EE0; font-size: 18px;">
    <strong>Why HiveMQ Requires TLS</strong>
  </span>
</div>

- MQTT without TLS usually uses port 1883 (insecure).

- HiveMQ Cloud only allows port 8883 with TLS.

- This protects your IoT system from spying, tampering, and fake brokers.

  
**Step 6: Example MicroPython Code for Pico W**

<div>
  <span style="color: #e49313; font-size: 16px;">
    <strong>secrets.py</strong>
  </span>
</div>

````python
import time

# ---------- Wi-Fi ----------
WIFI_SSID = "Your_WiFi_Name"
WIFI_PASS = "Your_WiFi_Password"

# ---------- MQTT (HiveMQ Cloud) ----------
MQTT_BROKER = "your-cluster-ID.s1.eu.hivemq.cloud"
MQTT_PORT   = 8883
MQTT_USER   = "pico_user1"
MQTT_PASS   = "StrongPassword123!"

# Unique-ish client id per boot
CLIENT_ID   = "pico-" + str(time.ticks_ms() & 0xffff)

````

<div>
  <span style="color: #e49313; font-size: 16px;">
    <strong>main.py</strong>
  </span>
</div>

```python
import network, time
from machine import Pin
from umqtt.simple import MQTTClient
import secrets

TOPIC_SUB = b"pico/led"
TOPIC_PUB = b"pico/status"

led = Pin(15, Pin.OUT)

# ---------- Wi-Fi connect ----------
wlan = network.WLAN(network.STA_IF)
wlan.active(True)
wlan.connect(secrets.WIFI_SSID, secrets.WIFI_PASS)

timeout_s = 15
t0 = time.time()
while not wlan.isconnected():
    if time.time() - t0 > timeout_s:
        raise RuntimeError("Wi-Fi connection timed out")
    time.sleep(0.2)

print("Wi-Fi connected:", wlan.ifconfig())

# ---------- Callback ----------
def sub_cb(topic, msg):
    print("MQTT Message:", topic, msg)

    if msg == b"ON":
        led.value(1)
        client.publish(TOPIC_PUB, b"LED is ON")

    elif msg == b"OFF":
        led.value(0)
        client.publish(TOPIC_PUB, b"LED is OFF")

# ---------- MQTT connect ----------
client = MQTTClient(
    client_id=secrets.CLIENT_ID,
    server=secrets.MQTT_BROKER,
    port=secrets.MQTT_PORT,
    user=secrets.MQTT_USER,
    password=secrets.MQTT_PASS,
    ssl=True,
    ssl_params={"server_hostname": secrets.MQTT_BROKER}
)

client.set_callback(sub_cb)
client.connect()
client.subscribe(TOPIC_SUB)

client.publish(TOPIC_PUB, b"Pico online")
print("Connected to HiveMQ Broker. Listening on", TOPIC_SUB)

# ---------- Main loop ----------
while True:
    client.check_msg()      # non-blocking
    time.sleep(0.2)

```

**Step 7: Testing Your Setup**

- Use the HiveMQ Web Client (built into the console):

    1- Go to your cluster dashboard → **Web Client**.
    
    2- Log in with the same MQTT username and password.
    
    3- Subscribe to `pico/status` to see status messages.
    
    4- Publish `"ON"` or `"OFF"` to the pico/led to control the Pico’s LED.
#### Resources

**MQTT.js** 

https://www.npmjs.com/package/mqtt

**JavaScript Getting Started Guide**

https://console.hivemq.cloud/clusters/5b0859fbe3274f67a2f6fecf1344e1a9/getting-started/clients/mqtt-js


## Task-1 “Greenhouse Guardian” — Publish DHT11 Sensor Data over MQTT
You’ve been hired to help a local urban greenhouse called Greenhouse Guardian. They grow herbs and succulents for cafés around the city. The grower wants live temperature and humidity readings from every bench so they can react quickly to heat waves or dry air. Your job: build a Pico W + DHT11 node that publishes readings to an MQTT broker every 10 seconds, and uses JSON so dashboards can consume it easily.

Your team will:

* Wire and read a DHT11 sensor
* Connect securely to an MQTT broker (HiveMQ Cloud—free)
* Publish {`"temp": 24.0, "hum": 52`} to a namespaced topic
* Prove it works using a web MQTT client (or phone app)
* Add a “device online/offline” Last Will message for reliability
#### Hardware Requirements
* Raspberry Pi Pico W
* DHT11 temperature & humidity sensor (3-pin or 4-pin module)
* Breadboard
* jumper wires
#### MQTT Broker Requirements (HiveMQ Cloud)
* A HiveMQ Cloud Serverless (free) cluster
* Hostname like abc123.s1.eu.hivemq.cloud
* Port 8883 (TLS)
* Username/Password credentials for your device

## Task-2 “Museum Heist Detector”
Your team has been hired by the National Tech Museum to create a smart security system for their new exhibition: *“Treasures of the Future”.* The artifacts on display are priceless, and the museum wants an IoT-based alarm

**System Requirements**

1. **Motion Detection** – Use a PIR sensor to detect intruders near the exhibit.
2. **Local Alarm** – Trigger a red LED (siren effect) and buzzer to scare off intruders.
3. **Remote Alerts** – Pico W publish real-time MQTT messages to the museum’s security control room.
4. **Remote Control** → Guards can send MQTT commands via phone/dashboard to:
    * `ARM` / `DISARM` the system
    * `MUTE` / `UNMUTE` the siren.
    
**Your Mission Brief**
You are the IoT Security Engineers.

* Install the PIR “laser tripwire” near the artifact.
* If anyone comes close while the system is **armed**, the alarm activates instantly.
* An **MQTT event message** is published to alert remote guards.
* Guards can respond by sending **MQTT commands** (e.g., `DISARM`, `MUTE`).
  
**Use Case Scenarios**

* During the day → system **Disarmed**, visitors enjoy exhibits.
* At night → system **Armed**, if motion is detected:
    * **Red LED** flashes like a siren light.
    * The **Buzzer sounds** loudly.
#### Hardware
* Raspberry Pi Pico W
* PIR motion sensor (HC-SR501 or similar)
* Red LED
* Buzzer
* Breadboard
* Jumpers
* Wi-Fi connection
  
#### Instructions
1. **Hardware Setup**

    * Connect the PIR sensor’s
    * Connect the LED (with resistor).
    * Connect the buzzer.
    * Ensure all components share GND.
      
2.**Wi-Fi & MQTT Setup**

* Configure the Pico W to connect to your Wi-Fi.
* Set up an MQTT broker (e.g., HiveMQ Cloud or a local Mosquitto broker).
* Define MQTT topics:
    * Publish: `museum/zone1/event`, `museum/zone1/state`
    * Subscribe: `museum/zone1/cmd`
      
3. **Programming Tasks**

* Write MicroPython code to:
    * Detect motion from PIR.
    * Flash LED and sound buzzer when the system is armed.
    * Publish an MQTT MOTION ALERT message.
    * Receive commands (`ARM`, `DISARM`, `MUTE`, `UNMUTE`) and update system behaviour.
      
4. **Testing the System**

* Run the code and verify Wi-Fi + MQTT connection.
* In MQTT Explorer or dashboard:
    * Send `ARM` → walk near the PIR → verify LED + buzzer + MQTT alert.
    * Send `MUTE` → LED flashes, buzzer stops.
    * Send `DISARM` → motion ignored, no alarm.